# Coordinates in Location Names
> How can the latitude and longitude be removed from location names?

## Description

[Long names, short names](01-long-names-short-names.ipynb) looked at the length of location
names. One option to shorten the names is to remove the coordinates added to every private
location. This notebook looks at what is needed to do that.

In [19]:
from django.db.models.functions import Length

from IPython.display import display, HTML

import matplotlib.pyplot as plt
import tabulate

from ebird.api.data.models import Location

In [67]:
# Location names, particularly private or one-time locations, end with the coordinates.

latitude = r"-?\d{1,2}[.,]\d{1,7}"
longitude = r"[-\u2212]?\d{1,3}[.,]\d{1,7}"
coordinates = r"\(?%s[,x] ?%s\)?" % (latitude, longitude)

total = Location.objects.all().count()
with_coords = Location.objects.filter(original__regex=coordinates)
without_coords = Location.objects.exclude(original__regex=coordinates)

labels = ["Coordinates", "No Coordinates", "Total"]
values = [with_coords.count(), without_coords.count(), total]
percentages = [round(value / total * 100, 1) for value in values]

data = zip(labels, values, percentages)

table = tabulate.tabulate(data, headers=["Category", "n", "%"], tablefmt='html')
table

Category,n,%
Coordinates,9231,50.2
No Coordinates,9166,49.8
Total,18397,100


In [68]:
# For the locations that did not match the regular expression for coordinates,
# display the ones that contain a decimal number. Use this to test how effective 
# the coorinates regular expression is.

locations = (
    without_coords
    .filter(original__regex=r"\d[.,]\d{2,}")
    .annotate(length=Length('original'))
    .order_by("-length")
)
names = [location.original for location in locations]

print("Count: %d\n" % locations.count())
print("\n".join(names))

Count: 33

10 Rua da Escola, Sobral de Monte Agraço, Lisboa, PT (39, -9,118)
3 Cova da Moura, Monfortinho, Castelo Branco, PT (40, -6.877)
CAC - NB50,60/P1 - Miradouro da Praia do Castelo
EM635, Carregal do Sal, Viseu, PT (40,408, -8)
Ponte Vasco da Gama, Setúbal, PT (38,732, -9)
CAC - NB50,60/P12 - Várzea de Armação de Pêra
Ponte Vasco da Gama, Setúbal, PT (38,732, -9)
CAC - NB50,60/P7 - Escolas de Vale Pedras
CAC - NB50,60/P19 - Caminho de Albandeira
Pego Das Pias, Beja, PT (37.645, ‎-8.619)
CAC - NB50,60/P15 - Vale de Alporchinhos
CAC - NB50,60/P4 - Parque ETAR Albufeira
Vale Da Lua, Beja, PT (37.709, ‎-8.491)
CAC - NB50,60/P8 - Caminhos da Mouraria
CAC - NB50,60/P13 - Lagoa dos Salgados
CAC - NB50,60/P16 - Areias de Porches
25 NM Sul Selvagem Pequena, -16.01387
CAC - NB50,60/P6 - Desvio das Areias
CAC - NB50,60/P2 - Pastoreio da Galé
CAC - NB50,60/P17 - Colina de Lombos
36º51.075'N, 08º52.410'W 2025-04-05
36º48.719'N, 08º54.417'W 2025-04-07
CAC - NB50,60/P11 - Areeiro de Pêra
ER243